In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/awayscreen/Looking Away/0106.jpg
/kaggle/input/awayscreen/Looking Away/0375.jpg
/kaggle/input/awayscreen/Looking Away/0285.jpg
/kaggle/input/awayscreen/Looking Away/0074.jpg
/kaggle/input/awayscreen/Looking Away/0077.jpg
/kaggle/input/awayscreen/Looking Away/0426.jpg
/kaggle/input/awayscreen/Looking Away/0235.jpg
/kaggle/input/awayscreen/Looking Away/0273.jpg
/kaggle/input/awayscreen/Looking Away/0444.jpg
/kaggle/input/awayscreen/Looking Away/0058.jpg
/kaggle/input/awayscreen/Looking Away/0255.jpg
/kaggle/input/awayscreen/Looking Away/0128.jpg
/kaggle/input/awayscreen/Looking Away/0364.jpg
/kaggle/input/awayscreen/Looking Away/0246.jpg
/kaggle/input/awayscreen/Looking Away/0337.jpg
/kaggle/input/awayscreen/Looking Away/0208.jpg
/kaggle/input/awayscreen/Looking Away/0141.jpg
/kaggle/input/awayscreen/Looking Away/0474.jpg
/kaggle/input/awayscreen/Looking Away/0372.jpg
/kaggle/input/awayscreen/Looking Away/0053.jpg
/kaggle/input/awayscreen/Looking Away/0355.jpg
/kaggle/input

In [2]:
import os
import cv2
import sys
import dlib
import argparse
import numpy as np
import pandas as pd

In [3]:
def drawPolyline(img, shapes, start, end, isClosed=False):
    points = []
    for i in range(start, end + 1):
        point = [shapes.part(i).x, shapes.part(i).y]
        points.append(point)
    points = np.array(points, dtype=np.int32)
    cv2.polylines(img, [points], isClosed, (255, 80, 0),
                  thickness=1, lineType=cv2.LINE_8)

def draw(img, shapes):
    drawPolyline(img, shapes, 0, 16)
    drawPolyline(img, shapes, 17, 21)
    drawPolyline(img, shapes, 22, 26)
    drawPolyline(img, shapes, 27, 30)
    drawPolyline(img, shapes, 30, 35, True)
    drawPolyline(img, shapes, 36, 41, True)
    drawPolyline(img, shapes, 42, 47, True)
    drawPolyline(img, shapes, 48, 59, True)
    drawPolyline(img, shapes, 60, 67, True)

In [4]:
def ref3DModel():
    modelPoints = [[0.0, 0.0, 0.0],
                   [0.0, -330.0, -65.0],
                   [-225.0, 170.0, -135.0],
                   [225.0, 170.0, -135.0],
                   [-150.0, -150.0, -125.0],
                   [150.0, -150.0, -125.0]]
    return np.array(modelPoints, dtype=np.float64)


def ref2dImagePoints(shape):
    imagePoints = [[shape.part(30).x, shape.part(30).y],
                   [shape.part(8).x, shape.part(8).y],
                   [shape.part(36).x, shape.part(36).y],
                   [shape.part(45).x, shape.part(45).y],
                   [shape.part(48).x, shape.part(48).y],
                   [shape.part(54).x, shape.part(54).y]]
    return np.array(imagePoints, dtype=np.float64)


def cameraMatrix(h,w):
    mat = [[1100, 1, h],
           [0, 1100, w],
           [0, 0, 1]]
    return np.array(mat, dtype=float)

In [5]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../input/shape-predictor-68-face-landmarksdhyani/shape_predictor_68_face_landmarks.dat')
def headposition(filepath):
    
    
    

    im = cv2.imread(filepath)

    faces = detector(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), 0)
    face3Dmodel = ref3DModel()


    for face in faces:
        shape = predictor(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), face)
        draw(im, shape)
        refImgPts = ref2dImagePoints(shape)
        height, width, channel = im.shape





        cameraMatrix1 = cameraMatrix(height/2,width/2)
        mdists = np.zeros((4, 1), dtype=np.float64)


        # calculate rotation and translation vector using solvePnP
        success, rotationVector, translationVector = cv2.solvePnP(face3Dmodel, refImgPts, cameraMatrix1, mdists)


        noseEndPoints3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
        noseEndPoint2D, jacobian = cv2.projectPoints(
        noseEndPoints3D, rotationVector, translationVector, cameraMatrix1, mdists)




        # calculating angle
        rmat, jac = cv2.Rodrigues(rotationVector)
        angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

        print("Angle: ", angles)
        #print(f"Qx:{Qx}\tQy:{Qy}\tQz:{Qz}\t")
        x = np.arctan2(Qx[2][1], Qx[2][2])
        y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
        z = np.arctan2(Qz[0][0], Qz[1][0])
#         print("AxisX: ", x)
#         print("AxisY: ", y)
#         print("AxisZ: ", z)
#         print('*' * 80)

        gaze = False        
        if angles[1] < -15:
            gaze = True         
        elif angles[1] > 15:
            gaze = True
        elif angles[0]<-173 :
            gaze =True
        else:
            gaze = False
            
        if(gaze==True):
            return 0
        else:
            return 1
        
        
        



In [6]:
my_list1=[]
file_path=[]

for subdir, dirs, files in os.walk('../input/awayscreen/Looking Away/'):
    cnt=0
    for file in files:
        filepath = subdir +  file
        print(filepath)
        file_path.append(filepath)
        my_list1.append(headposition(filepath))
        
        

../input/awayscreen/Looking Away/0106.jpg
Angle:  (-155.48204570586068, 48.24812134061437, 16.848425453577082)
../input/awayscreen/Looking Away/0375.jpg
../input/awayscreen/Looking Away/0285.jpg
Angle:  (-160.8741786224802, -11.163872328899018, -4.589895445494236)
../input/awayscreen/Looking Away/0074.jpg
Angle:  (-161.95317723013568, 41.865843345305464, 14.865869969875474)
../input/awayscreen/Looking Away/0077.jpg
Angle:  (-160.461522201406, 42.57159304518102, 16.188538493129276)
../input/awayscreen/Looking Away/0426.jpg
Angle:  (-142.13496986247426, 46.196093169546565, 24.144077164030424)
../input/awayscreen/Looking Away/0235.jpg
Angle:  (134.68590531464244, -61.31574372268382, -141.73132860871218)
../input/awayscreen/Looking Away/0273.jpg
Angle:  (-159.03836798103606, -19.0457742517796, -4.835019769244155)
../input/awayscreen/Looking Away/0444.jpg
Angle:  (-151.11031633512192, 48.81743194080457, 26.416293170583934)
../input/awayscreen/Looking Away/0058.jpg
Angle:  (-174.059521922507

In [7]:
my_list2=[]
file_path=[]

for subdir, dirs, files in os.walk('../input/towardsscreen/Looking Towards/'):
    cnt=0
    for file in files:
        filepath = subdir +  file
        print(filepath)
        file_path.append(filepath)
        my_list2.append(headposition(filepath))
        

../input/towardsscreen/Looking Towards/0106.jpg
Angle:  (-157.23041840322648, 19.933252640837242, 1.7489546642649758)
../input/towardsscreen/Looking Towards/0375.jpg
Angle:  (-160.55762643047646, 22.324147339273072, 3.5081447341530034)
../input/towardsscreen/Looking Towards/0285.jpg
Angle:  (-166.37824058918045, -1.4022504175775403, -2.1471448272990394)
../input/towardsscreen/Looking Towards/0074.jpg
Angle:  (-156.83180980813188, 7.795815267068296, -1.9829056131785334)
../input/towardsscreen/Looking Towards/0077.jpg
Angle:  (-151.1678559987313, 7.727565798534549, -1.1691400043989701)
../input/towardsscreen/Looking Towards/0498.jpg
Angle:  (-158.8772157744048, 21.481296216228845, 3.3748154418827263)
../input/towardsscreen/Looking Towards/0426.jpg
Angle:  (-152.868161055101, 20.90646054330926, 1.1670331007420889)
../input/towardsscreen/Looking Towards/0235.jpg
Angle:  (-165.41523996977364, 4.567714314734521, 3.0646039926991664)
../input/towardsscreen/Looking Towards/0273.jpg
Angle:  (-16

In [8]:
print(my_list1)

[0, None, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, None, 0, None, None, None, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, None, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 0, 0, 0, 1, 1, 0, 0, 0, None, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, None, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 

In [9]:
print(my_list2)

[0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 